<a href="https://colab.research.google.com/github/fernand0aguilar/peregrine/blob/master/Train_a_GPT_2_Model_on_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses a Nvidia T4 GPU, Nvidia K80 GPU, or a P100 GPU. The P100 is ideal. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Sat Dec  5 13:29:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.

Using the `774M` or `1558M` models is not recommended with tweet data, as tweets are short.

You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 272Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 81.7Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 302Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:05, 251Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 141Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 42.8Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 175Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "Fernand0Aguilar_history.csv"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=2000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=500,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 274.21it/s]

Loading dataset...
dataset has 1498 tokens
Training...


[10 | 24.65] loss=1.43 avg=1.43
[20 | 40.28] loss=0.50 avg=0.96
[30 | 56.22] loss=0.08 avg=0.67
[40 | 72.53] loss=0.10 avg=0.52
[50 | 89.20] loss=0.02 avg=0.42
[60 | 106.41] loss=0.01 avg=0.35
[70 | 123.66] loss=0.02 avg=0.30
[80 | 140.52] loss=0.01 avg=0.26
[90 | 157.25] loss=0.01 avg=0.23
[100 | 174.07] loss=0.00 avg=0.21
[110 | 191.08] loss=0.01 avg=0.19
[120 | 208.14] loss=0.01 avg=0.18
[130 | 225.10] loss=0.01 avg=0.16
[140 | 241.94] loss=0.01 avg=0.15
[150 | 258.79] loss=0.01 avg=0.14


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

However, for tweets, generating from the base model alone won't be as fruitful due to the necessary encoding. For a solution to that, see the next cell.

In [ ]:
gpt2.generate(sess, run_name='run1')

by

I WILL NEVER CHANGE THIS<|endoftext|>
<|startoftext|>😩❤️<|endoftext|>
<|startoftext|>I definitely will be doing some of those things! I'm not sure where I'll live but I'll be there!<|endoftext|>
<|startoftext|>I have a paunchy, booty blonde haired boy<|endoftext|>
<|startoftext|>I like it when people call me out for being ignorant and rude. That is my form of self awareness.<|endoftext|>
<|startoftext|>The only thing worse in this day is a fog machine, I swear<|endoftext|>
<|startoftext|>I don't know how to do the "this won't work on my phone" comments. I'll play save the world by a cry in the elevator or on the plane<|endoftext|>
<|startoftext|>I have no fucking idea how I'm supposed to look on this plane with my sunglasses on. I am blind. I am blind.<|endoftext|>
<|startoftext|>I just tried to get the 's voice to come through. I thought it would be a little boy but it wasn't? I don't get it. I'm blind.<|endoftext|>
<|startoftext|>I am so confused as to why anyone would send someo

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

Tweets require specific `prefix`, `truncate`, and `include_prefix` as demoed below.

In [ ]:
gpt2.generate(sess,
              length=200,
              temperature=0.7,
              prefix='<|startoftext|>',
              truncate='<|endoftext|>',
              include_prefix=False,
              nsamples=20,
              batch_size=20
              )

What are you doing in an empty hotel cubicle? Do u think it's the hotel?
Yes. I just had that same thought and voila. She does not. She is an idiot.
lol
lol
hahahaha
I love him. RT :  My favorite Twitter tweet of the season.
I've been listening to this train of thought for way too long. I'm lost.
❤️
I'm not a good negotiator. I don't need you to tell me I can't have a fucking adult conversation about stupid shit.
i am gonna kill myself by telling  to get out of my face?
This is a really bad tweet. I was hoping he said sorry and I was mad I didn't say hi.
I'm in. I'm in.
I'm not making this up. I can't stop typing.
lol!
all the same. great tweet. oh boy oh boy.
I wrote to you! I wanted to buy the book. I love you guys so much
I love you, Sarah
I'm like "oo this looks awesome" and then it is like "no shit I'm not sure I want this" and I'm like "no it's not a fucking book that's what u said" and I'm like oh shit no it's a fucking cheetos commercial
I just went to my first martial arts cla

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=200,
                      temperature=1.0,
                      top_p=0.9,
                      prefix='<|startoftext|>',
                      truncate='<|endoftext|>',
                      include_prefix=False,
                      nsamples=1000,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

If you want to generate a large number of 1,000 tweet files without having to click a cell button multiple times, you can use the cell below. You can then download the generated files by right-clicking them in the Files sidebar.

In [ ]:
num_files = 10

for _ in range(num_files):
  gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

  gpt2.generate_to_file(sess,
                        destination_path=gen_file,
                        length=200,
                        temperature=1.0,
                        top_p=0.9,
                        prefix='<|startoftext|>',
                        truncate='<|endoftext|>',
                        include_prefix=False,
                        nsamples=1000,
                        batch_size=20
                        )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019-2020 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.